# Case Study: Molecular Dynamics Simulations of Water

Water, with its simple molecular structure, exhibits complex behavior and interactions due to the formation of hydrogen bonds. 
These interactions can be studied in detail at the atomic and molecular level through molecular dynamics (MD) simulations. 
In this case study, we employ the OpenMM software library to simulate a box of water molecules, tracking their movements and interactions over time.

Starting with the input files (prmtop and inpcrd), we first define the conditions and parameters of our simulation environment. The system undergoes an equilibration phase under NPT (constant Number of particles, Pressure, and Temperature) conditions followed by a subsequent equilibration under NVT (constant Number of particles, Volume, and Temperature) conditions. Once the system is adequately equilibrated, a production run is performed, during which the trajectory of each water molecule is recorded for detailed analysis.

Following the simulation, we analyze the radial distribution functions (RDFs) for the Oxygen-Oxygen (O-O) and Oxygen-Hydrogen (O-H) pairs, providing insights into the short-range order and hydrogen bonding patterns in the water box.

In [ ]:
import openmm as mm
from openmm import app, unit, LangevinIntegrator, MonteCarloBarostat

import nglview as nv

import time

In [ ]:

# Load the prmtop and inpcrd files
prmtop = app.AmberPrmtopFile("water/water_box.prmtop")
inpcrd = app.AmberInpcrdFile("water/water_box.inpcrd")

# Specify the GPU platform
try:
    platform = mm.Platform.getPlatformByName('CUDA')
    properties = {'CudaPrecision': 'mixed'} 
except mm.OpenMMException:
    platform = mm.Platform.getPlatformByName('Reference')

In [ ]:
# Create the system
system = prmtop.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometers, constraints=app.HBonds)

start = time.time()

# NPT equilibration
temperature = 300 * unit.kelvin
pressure = 1 * unit.atmosphere
integratorNPT = LangevinIntegrator(temperature, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)
system.addForce(MonteCarloBarostat(pressure, temperature))
simulationNPT = app.Simulation(prmtop.topology, system, integratorNPT)
simulationNPT.context.setPositions(inpcrd.positions)

if inpcrd.boxVectors is not None:
    simulationNPT.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

simulationNPT.minimizeEnergy()
simulationNPT.reporters.append(app.StateDataReporter("water/npt.log", 1000, step=True, potentialEnergy=True, temperature=True, density=True))
simulationNPT.step(50000)  # NPT equilibration for 100 ps
end_npt = time.time()

# Get final equilibration state
state_eq = simulationNPT.context.getState(getPositions=True, getVelocities=True, enforcePeriodicBox=True)
positions_eq = state_eq.getPositions()
velocities_eq = state_eq.getVelocities()
box_vectors_eq = state_eq.getPeriodicBoxVectors()


In [ ]:
# Set up production run
integratorProd = LangevinIntegrator(temperature, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)
simulationProd = app.Simulation(prmtop.topology, system, integratorProd)

# Set positions and velocities to those from equilibration run
simulationProd.context.setPositions(positions_eq)
simulationProd.context.setVelocities(velocities_eq)

# Set periodic box vectors to those from equilibration run
simulationProd.context.setPeriodicBoxVectors(*box_vectors_eq)

# Add simulation reporters
simulationProd.reporters.append(app.DCDReporter("water/production.dcd", 1000))  # Save trajectory
simulationProd.reporters.append(app.StateDataReporter("water/production.log", 1000, step=True, potentialEnergy=True, temperature=True, density=True))

# Run production simulation
simulationProd.step(500000) # Production run for 1 ns

end_production = time.time()


In [ ]:
# Print timings
print("Simulation completed!")
print(f"Total Time:{end_production - start}" )
print(f"Equilibration 1: :{end_npt - start}")
print(f"Production: {end_production - end_npt}")

## Visualization

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import rdf
import matplotlib.pyplot as plt

# Load the trajectory
u = mda.Universe("water/water_box.prmtop", "water/production.dcd")

view = nv.show_mdanalysis(u)

# Clear default representation
view.clear_representations()

# Add a new representation
view.add_representation("ball+stick")

view.camera = "orthographic"
view.center()

view

## Analysis

Calculation of RDF for water system.

In [ ]:

# Define the atom groups for oxygen and hydrogen
oxygen_atoms = u.select_atoms("element O")  
hydrogen_atoms = u.select_atoms("element H") 

# Calculate the O-O RDF
rdf_OO = rdf.InterRDF(oxygen_atoms, oxygen_atoms, range=(0.0, 10.0), bins=100, exclude_same="residue")
rdf_OO.run()
                                                                                                                                                                                                
# Calculate the O-H RDF
rdf_OH = rdf.InterRDF(oxygen_atoms, hydrogen_atoms, range=(0.0, 10.0), bins=100, exclude_same="residue")
rdf_OH.run()




In [ ]:
# Plotting
plt.figure(figsize=(10, 4))

# O-O RDF
plt.subplot(1, 2, 1)
plt.plot(rdf_OO.results.bins, rdf_OO.results.rdf)
plt.title("O-O RDF")
plt.xlabel("Distance (Å)")
plt.ylabel("g(r)")
plt.grid(True)

# O-H RDF
plt.subplot(1, 2, 2)
plt.plot(rdf_OH.results.bins, rdf_OH.results.rdf)
plt.title("O-H RDF")
plt.xlabel("Distance (Å)")
plt.ylabel("g(r)")
plt.grid(True)

plt.tight_layout()